In [1]:
from secret_key import OPENAI_API_KEY
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.6)
restaurantName = llm("I want to open an Indian Restaurant in Boston Massachusetts. Can you help me with a quirky name? ")
print(restaurantName)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



1. "Curry Up and Spice"
2. "Tandoori Tales"
3. "Naan Stop Indian Kitchen"
4. "Masala Madness"
5. "Chai and Chaat Co."
6. "Samosa Shack"
7. "Bollywood Bites"
8. "Flavors of India"
9. "Namaste Kitchen"
10. "The Spice Route Eatery"


In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template_restaurant_name = PromptTemplate(
    input_variables= ['cuisine'],
    template= 'I want to open an {cuisine} Restaurant in Boston Massachusetts. Can you help me with a quirky name?'
)

name_chain = LLMChain(llm = llm, prompt = prompt_template_restaurant_name)

prompt_template_items = PromptTemplate(
    input_variables= ['restaurant_name'],
    template= 'Suggest some menu items for {restaurant_name}. Return it as a Comma Seperated list'
)

food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
# Simple Sequential Chain
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Mexican")
print(response)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




1. "Taco Tumble": Chicken, beef, or veggie tacos with choice of toppings such as salsa, guacamole, sour cream, and cheese.
2. "Burrito Bazaar": Build-your-own burrito with choice of protein (chicken, beef, or pork), rice, beans, cheese, and toppings.
3. "Salsa Sensations": Trio of house-made salsas (mild, medium, and hot) served with chips.
4. "Enchilada Emporium": Choice of chicken, beef, or cheese enchiladas topped with red or green sauce and cheese.
5. "Quesadilla Quirks": Traditional cheese quesadilla or choice of protein (chicken, beef, or veggie) with added toppings such as peppers, onions, and black beans.
6. "Fiesta Frenzy": Combination platter with a variety of Mexican favorites such as tacos, burritos, and enchiladas.
7. "Chimichanga Charm": Fried flour tortilla filled with choice of protein (chicken, beef, or pork), rice, beans, and cheese.
8. "Nacho Nation": Loaded nachos with choice of protein (chicken, beef


In [5]:
# Sequential Chain

# Adding output keys to name_chain and food_items_chain

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template_restaurant_name = PromptTemplate(
    input_variables= ['cuisine'],
    template= 'I want to open an {cuisine} Restaurant in Boston Massachusetts. Can you help me with a quirky name?'
)

name_chain = LLMChain(llm = llm, prompt = prompt_template_restaurant_name, output_key= "restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables= ['restaurant_name'],
    template= 'Suggest some menu items for {restaurant_name}. Return it as a Comma Seperated list'
)

food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items,output_key= "menu_items")



In [6]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ["cuisine"],
    output_variables = ['restaurant_name', 'menu_items']
)

chain({'cuisine':'Indian'})

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Spice Odyssey: A Taste of India in Boston"',
 'menu_items': '\n\n1. Samosas\n2. Butter Chicken\n3. Tandoori Chicken\n4. Palak Paneer\n5. Vegetable Biryani\n6. Naan Bread\n7. Chicken Tikka Masala\n8. Mango Lassi\n9. Aloo Gobi\n10. Chana Masala\n11. Lamb Vindaloo\n12. Garlic Naan\n13. Dal Makhani\n14. Malai Kofta\n15. Rogan Josh\n16. Masala Chai\n17. Kulfi\n18. Bhindi Masala\n19. Chicken Korma\n20. Gajar Halwa'}

In [17]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["wikipedia","llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

agent.run("When was Elon Musk born? What will his age be in 2030")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and then use the Calculator to calculate his age in 2030.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman and investor known for his key roles in space company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp., formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI. He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be US$221 billion.
Musk was born in Pretoria to Maye and Errol Musk and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and r

'Elon Musk will be 59 years old in 2030.'

In [10]:
from secret_key import SERP_API_KEY

os.environ['SERPAPI_API_KEY'] = SERP_API_KEY

In [13]:
llm = OpenAI(temperature=0)

tools = load_tools(["serpapi","llm-math"], llm = llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

agent.run("What was the GDP of US in 2022, plus 5?")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to perform the math.
Action: Calculator
Action Input: 2022 + 5
Observation: Answer: 2027
Thought: Now I need to search for the GDP of the US in 2027.
Action: Search
Action Input: "GDP of US in 2027"
Observation: {'type': 'organic_result', 'title': 'United States - Gross domestic product (GDP) 2029 - Statista', 'source': 'Statista'}
Thought: I now know the final answer.
Final Answer: The GDP of the US in 2027 is $24.5 trillion.

> Finished chain.


'The GDP of the US in 2027 is $24.5 trillion.'

In [14]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt = prompt_template_restaurant_name, memory = memory)
restaurantName = chain.run("Mexican")
print(restaurantName)



1. "Taco Tumble"
2. "Salsa Sensations"
3. "Burrito Bonanza"
4. "Enchilada Emporium"
5. "Quesadilla Quirks"
6. "Fiesta Frenzy"
7. "Guacamole Galore"
8. "Chimichanga Charm"
9. "Nacho Nation"
10. "Tortilla Twist"


In [16]:
print(chain.memory.buffer)

Human: Mexican
AI: 

1. "Taco Tumble"
2. "Salsa Sensations"
3. "Burrito Bonanza"
4. "Enchilada Emporium"
5. "Quesadilla Quirks"
6. "Fiesta Frenzy"
7. "Guacamole Galore"
8. "Chimichanga Charm"
9. "Nacho Nation"
10. "Tortilla Twist"


In [1]:
from langchain_helper import test

test()

hello
